In [2]:
import dask.array as da
import numpy as np

In [3]:
X = da.asarray(np.random.randn(10,10,10, 10, 10), chunks=(1,2,3, 4, 5))
X

dask.array<array, shape=(10, 10, 10, 10, 10), dtype=float64, chunksize=(1, 2, 3, 4, 5), chunktype=numpy.ndarray>

In [4]:
chunksize = X.chunksize
chunksize_numpy = np.array((chunksize))
chunksize_numpy

array([1, 2, 3, 4, 5])

In [5]:
chunksize_numpy[[2,3,4]] = -1

In [6]:
tuple(chunksize_numpy)

(1, 2, -1, -1, -1)

In [7]:
len(X.shape)

5

In [8]:
X.dtype

dtype('float64')

In [9]:
np.array([1,2,3,4.5]).dtype

dtype('float64')

In [10]:
X.ndim

5

In [11]:
from tools import FourierTools

In [12]:
X_fft = FourierTools.cartesian_FT_dask(array=X, direction="forward", fft_shift=True, axes=(2,3,4), device="cpu")
X_fft

[  1  ][ success ] ---v  (collected several lines)                      
                      Adding Fourier Transformation to Dask Graph:
                       => Direction:............... forward
                       => Applying (i)FFT-shift:... True
                       => Affected axes:........... (2, 3, 4) of 5 total axes
                       => Rechunking:.............. (1, 2, 3, 4, 5) => (1, 2, 10, 10, 10)
                       => Data type:............... float64 => <class 'numpy.complex128'>
                       => Performing on device:.... cpu
                      


dask.array<fftshift, shape=(10, 10, 10, 10, 10), dtype=complex128, chunksize=(1, 2, 10, 10, 10), chunktype=numpy.ndarray>

In [14]:
X_fft.shape[2:]

(10, 10, 10)